In [1]:
%gui qt
# Load modules
import Motion
import numpy as np
from numpy.random import randn
# Vispy ------------------------
import vispy
from vispy.scene import visuals
from vispy import app

# Generate initial conditions 

Use `Motion.rsphere(Motion.sphere(r, N), w=noise)` to generate a noisy sphere

### Single body

In [12]:
R = Motion.rsphere(Motion.sphere(r=4, N=300), w=0.5)
P = Motion.unitary(randn(len(R),3))
Q = Motion.unitary(randn(len(R),3))*0

### Two-body problem

In [9]:
R1 = Motion.sphere(r=2, N=100)
R2 = Motion.sphere(r=2, N=100) - 6
R = np.concatenate((R1,R2), axis=0)
P = Motion.unitary(randn(len(R),3))
Q = Motion.unitary(randn(len(R),3))

### Visualise initial contitions

In [13]:
frame = vispy.scene.SceneCanvas(title='Initial conditions', 
                                keys='interactive', bgcolor='black', show=True)
pov = frame.central_widget.add_view()
pov.camera = 'turntable'

vec = visuals.Markers(scaling=True, spherical=True, symbol='disc', alpha=1)
abp = visuals.Arrow(arrow_type='triangle_30', antialias=True, connect="segments", width=1)
pcp = visuals.Arrow(arrow_type='triangle_30', antialias=True, connect="segments", width=1)

pov.add(vec)
pov.add(abp)
pov.add(pcp)
pov.camera.fov = 45
pov.camera.distance = 15

vec.set_data(R, size=100, edge_width=0, face_color="#237DBA", edge_color="white")
abp.set_data(np.stack((R, R+P), axis=1), color="#A31368")
pcp.set_data(np.stack((R, R+Q), axis=1), color="#60EC83")

if __name__ == '__main__':
    frame.show()
    app.run()

# Simulation

## Strength of attractive/repulsive potential

Calculate the weights of the attraction coefficients according to the formulae:

\begin{equation}
\begin{split}
S_1 &= (p^i \wedge r^{ij}) \cdot (p^j \wedge r^{ij}) = p^{i}_{\mu} p^{j}_{\mu} r^{ij}_{\mu}r^{ij}_{\mu} - p^{i}_{\mu}r^{ij}_{\mu} p^{j}_{\nu}r^{ij}_{\nu} \\
S_2 &= (p^i \wedge q^{i}) \cdot (p^j \wedge q^{j}) = p^{i}_{\mu} p^{j}_{\mu} q^{i}_{\nu}q^{j}_{\nu} - p^{i}_{\mu}q^{j}_{\mu} p^{j}_{\nu}q^{i}_{\nu} \\
S_3 &= (q^i \wedge r^{ij}) \cdot (q^j \wedge r^{ij}) = q^{i}_{\mu} q^{j}_{\mu} r^{ij}_{\mu}r^{ij}_{\mu} - q^{i}_{\mu}r^{ij}_{\mu} q^{j}_{\nu}r^{ij}_{\nu} \\
\end{split}
\end{equation}

The strength of attraction is $S$, where
\begin{equation}
S = \sigma_1 S_1 + \sigma_2 S_2 + \sigma_3 S_3, \qquad \sigma_1 + \sigma_2 + \sigma_3 = 1.
\end{equation}

## Dynamical model - Brownian dynamics

\begin{equation}
\frac{\mathrm{d}X}{\mathrm{d}t} = -\frac{D}{k_BT}\nabla \Phi(X) + \sqrt{2D}R(t),
\end{equation}

for numerical purposes all constants have been set to unit.

## Control panel

In [5]:
import Simulation

In [14]:
ITERS = 2000
s = Simulation.Evolution(R, P, Q, ITERS, sigma=[0.8, 0.1, 0.1], video=False)
time = app.Timer()
time.connect(s.update)
time.start(interval=0, iterations=ITERS)
time.events.stop.connect(lambda x: app.quit())

if __name__ == '__main__':
    s.show()
    app.run()

## TODO

> Radiation

> Strain tensor